In [1]:
from __future__ import print_function

import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import keras
from keras.models import Sequential, Model, load_model
from keras import backend as K

import tensorflow as tf
'''
from keras.backend.tensorflow_backend import set_session

def contain_tf_gpu_mem_usage() :
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)
    set_session(sess)

contain_tf_gpu_mem_usage()
'''
import os
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.cm as cm
#import seaborn as sns

import urllib
import urllib.request
import pickle
from time import sleep

from scipy.stats import ttest_ind
from scipy.stats import pearsonr, spearmanr

from sklearn.linear_model import LinearRegression

from scipy.optimize import minimize

import h5py

#Visualization code

import matplotlib

def ic_scale(pwm,background):
    per_position_ic = util.compute_per_position_ic(
                       ppm=pwm, background=background, pseudocount=0.001)
    return pwm*(per_position_ic[:,None])


def plot_a(ax, base, left_edge, height, color):
    a_polygon_coords = [
        np.array([
           [0.0, 0.0],
           [0.5, 1.0],
           [0.5, 0.8],
           [0.2, 0.0],
        ]),
        np.array([
           [1.0, 0.0],
           [0.5, 1.0],
           [0.5, 0.8],
           [0.8, 0.0],
        ]),
        np.array([
           [0.225, 0.45],
           [0.775, 0.45],
           [0.85, 0.3],
           [0.15, 0.3],
        ])
    ]
    for polygon_coords in a_polygon_coords:
        ax.add_patch(matplotlib.patches.Polygon((np.array([1,height])[None,:]*polygon_coords
                                                 + np.array([left_edge,base])[None,:]),
                                                facecolor=color, edgecolor=color))


def plot_c(ax, base, left_edge, height, color):
    ax.add_patch(matplotlib.patches.Ellipse(xy=[left_edge+0.65, base+0.5*height], width=1.3, height=height,
                                            facecolor=color, edgecolor=color))
    ax.add_patch(matplotlib.patches.Ellipse(xy=[left_edge+0.65, base+0.5*height], width=0.7*1.3, height=0.7*height,
                                            facecolor='white', edgecolor='white'))
    ax.add_patch(matplotlib.patches.Rectangle(xy=[left_edge+1, base], width=1.0, height=height,
                                            facecolor='white', edgecolor='white', fill=True))


def plot_g(ax, base, left_edge, height, color):
    ax.add_patch(matplotlib.patches.Ellipse(xy=[left_edge+0.65, base+0.5*height], width=1.3, height=height,
                                            facecolor=color, edgecolor=color))
    ax.add_patch(matplotlib.patches.Ellipse(xy=[left_edge+0.65, base+0.5*height], width=0.7*1.3, height=0.7*height,
                                            facecolor='white', edgecolor='white'))
    ax.add_patch(matplotlib.patches.Rectangle(xy=[left_edge+1, base], width=1.0, height=height,
                                            facecolor='white', edgecolor='white', fill=True))
    ax.add_patch(matplotlib.patches.Rectangle(xy=[left_edge+0.825, base+0.085*height], width=0.174, height=0.415*height,
                                            facecolor=color, edgecolor=color, fill=True))
    ax.add_patch(matplotlib.patches.Rectangle(xy=[left_edge+0.625, base+0.35*height], width=0.374, height=0.15*height,
                                            facecolor=color, edgecolor=color, fill=True))


def plot_t(ax, base, left_edge, height, color):
    ax.add_patch(matplotlib.patches.Rectangle(xy=[left_edge+0.4, base],
                  width=0.2, height=height, facecolor=color, edgecolor=color, fill=True))
    ax.add_patch(matplotlib.patches.Rectangle(xy=[left_edge, base+0.8*height],
                  width=1.0, height=0.2*height, facecolor=color, edgecolor=color, fill=True))

default_colors = {0:'green', 1:'blue', 2:'orange', 3:'red'}
default_plot_funcs = {0:plot_a, 1:plot_c, 2:plot_g, 3:plot_t}
def plot_weights_given_ax(ax, array,
                 figsize=(20,2),
                 height_padding_factor=0.2,
                 length_padding=1.0,
                 subticks_frequency=1.0,
                 colors=default_colors,
                 plot_funcs=default_plot_funcs,
                 highlight={},
                 ylabel=""):
    if len(array.shape)==3:
        array = np.squeeze(array)
    assert len(array.shape)==2, array.shape
    if (array.shape[0]==4 and array.shape[1] != 4):
        array = array.transpose(1,0)
    assert array.shape[1]==4
    max_pos_height = 0.0
    min_neg_height = 0.0
    heights_at_positions = []
    depths_at_positions = []
    for i in range(array.shape[0]):
        #sort from smallest to highest magnitude
        acgt_vals = sorted(enumerate(array[i,:]), key=lambda x: abs(x[1]))
        positive_height_so_far = 0.0
        negative_height_so_far = 0.0
        for letter in acgt_vals:
            plot_func = plot_funcs[letter[0]]
            color=colors[letter[0]]
            if (letter[1] > 0):
                height_so_far = positive_height_so_far
                positive_height_so_far += letter[1]                
            else:
                height_so_far = negative_height_so_far
                negative_height_so_far += letter[1]
            plot_func(ax=ax, base=height_so_far, left_edge=i, height=letter[1], color=color)
        max_pos_height = max(max_pos_height, positive_height_so_far)
        min_neg_height = min(min_neg_height, negative_height_so_far)
        heights_at_positions.append(positive_height_so_far)
        depths_at_positions.append(negative_height_so_far)

    #now highlight any desired positions; the key of
    #the highlight dict should be the color
    for color in highlight:
        for start_pos, end_pos in highlight[color]:
            assert start_pos >= 0.0 and end_pos <= array.shape[0]
            min_depth = np.min(depths_at_positions[start_pos:end_pos])
            max_height = np.max(heights_at_positions[start_pos:end_pos])
            ax.add_patch(
                matplotlib.patches.Rectangle(xy=[start_pos,min_depth],
                    width=end_pos-start_pos,
                    height=max_height-min_depth,
                    edgecolor=color, fill=False))
            
    ax.set_xlim(-length_padding, array.shape[0]+length_padding)
    ax.xaxis.set_ticks(np.arange(0.0, array.shape[0]+1, subticks_frequency))
    height_padding = max(abs(min_neg_height)*(height_padding_factor),
                         abs(max_pos_height)*(height_padding_factor))
    ax.set_ylim(min_neg_height-height_padding, max_pos_height+height_padding)
    ax.set_ylabel(ylabel)
    ax.yaxis.label.set_fontsize(15)


def plot_weights(array,
                 figsize=(20,2),
                 **kwargs):
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111) 
    plot_weights_given_ax(ax=ax, array=array,**kwargs)
    plt.show()


def plot_score_track_given_ax(arr, ax, threshold=None, **kwargs):
    ax.plot(np.arange(len(arr)), arr, **kwargs)
    if (threshold is not None):
        ax.plot([0, len(arr)-1], [threshold, threshold])
    ax.set_xlim(0,len(arr)-1)

from matplotlib.ticker import FormatStrFormatter

def plot_score_track(arr, threshold=None, figsize=(20,2), **kwargs):
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111) 
    plot_score_track_given_ax(arr, threshold=threshold, ax=ax, **kwargs) 
    plt.show()

def plot_pwm(weights, figsize=(16, 2), plot_y_ticks=True, y_min=None, y_max=None, save_figs=False, fig_name="default") :
    colors = {0:'green', 1:'blue', 2:'orange', 3:'red'}
    
    plot_funcs = {0: plot_a, 1: plot_c, 
                  2: plot_g, 3: plot_t}

    fig = plt.figure(figsize=figsize)
    
    ax = fig.add_subplot(111) 
    
    plot_weights_given_ax(ax=ax, array=weights, 
                                       height_padding_factor=0.2,
                                       length_padding=1.0, 
                                       subticks_frequency=1.0, 
                                       colors=colors, plot_funcs=plot_funcs, 
                                       highlight={}, ylabel="")

    plt.sca(ax)
    
    plt.xticks([], [])
    plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
    
    if plot_y_ticks :
        plt.yticks(fontsize=12)
    else :
        plt.yticks([], [])
    
    if y_min is not None and y_max is not None :
        plt.ylim(y_min, y_max)
    elif y_min is not None :
        plt.ylim(y_min)
    
    plt.tight_layout()
    
    if save_figs :
        plt.savefig(fig_name + ".png", transparent=True, dpi=600)
        plt.savefig(fig_name + ".eps")
    
    plt.show()

import matplotlib.cm as cm
import matplotlib.colors as colors

import matplotlib as mpl
from matplotlib.text import TextPath
from matplotlib.patches import PathPatch, Rectangle
from matplotlib.font_manager import FontProperties
from matplotlib import gridspec
from matplotlib.ticker import FormatStrFormatter

def dna_letter_at(letter, x, y, yscale=1, ax=None, color=None, alpha=1.0):

    fp = FontProperties(family="DejaVu Sans", weight="bold")
    globscale = 1.35
    LETTERS = {	"T" : TextPath((-0.305, 0), "T", size=1, prop=fp),
                "G" : TextPath((-0.384, 0), "G", size=1, prop=fp),
                "A" : TextPath((-0.35, 0), "A", size=1, prop=fp),
                "C" : TextPath((-0.366, 0), "C", size=1, prop=fp),
                "UP" : TextPath((-0.488, 0), '$\\Uparrow$', size=1, prop=fp),
                "DN" : TextPath((-0.488, 0), '$\\Downarrow$', size=1, prop=fp),
                "(" : TextPath((-0.25, 0), "(", size=1, prop=fp),
                "." : TextPath((-0.125, 0), "-", size=1, prop=fp),
                ")" : TextPath((-0.1, 0), ")", size=1, prop=fp)}
    COLOR_SCHEME = {'G': 'orange',#'orange', 
                    'A': 'green',#'red', 
                    'C': 'blue',#'blue', 
                    'T': 'red',#'darkgreen',
                    'UP': 'green', 
                    'DN': 'red',
                    '(': 'black',
                    '.': 'black', 
                    ')': 'black'}


    text = LETTERS[letter]

    chosen_color = COLOR_SCHEME[letter]
    if color is not None :
        chosen_color = color

    t = mpl.transforms.Affine2D().scale(1*globscale, yscale*globscale) + \
        mpl.transforms.Affine2D().translate(x,y) + ax.transData
    p = PathPatch(text, lw=0, fc=chosen_color, alpha=alpha, transform=t)
    if ax != None:
        ax.add_artist(p)
    return p

def plot_seq_scores(importance_scores, figsize=(16, 2), plot_y_ticks=True, y_min=None, y_max=None, save_figs=False, fig_name="default") :

    importance_scores = importance_scores.T

    fig = plt.figure(figsize=figsize)
    
    ref_seq = ""
    for j in range(importance_scores.shape[1]) :
        argmax_nt = np.argmax(np.abs(importance_scores[:, j]))
        
        if argmax_nt == 0 :
            ref_seq += "A"
        elif argmax_nt == 1 :
            ref_seq += "C"
        elif argmax_nt == 2 :
            ref_seq += "G"
        elif argmax_nt == 3 :
            ref_seq += "T"

    ax = plt.gca()
    
    for i in range(0, len(ref_seq)) :
        mutability_score = np.sum(importance_scores[:, i])
        color = None
        dna_letter_at(ref_seq[i], i + 0.5, 0, mutability_score, ax, color=color)
    
    plt.sca(ax)
    plt.xticks([], [])
    plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
    
    plt.xlim((0, len(ref_seq)))
    
    #plt.axis('off')
    
    if plot_y_ticks :
        plt.yticks(fontsize=12)
    else :
        plt.yticks([], [])
    
    if y_min is not None and y_max is not None :
        plt.ylim(y_min, y_max)
    elif y_min is not None :
        plt.ylim(y_min)
    else :
        plt.ylim(
            np.min(importance_scores) - 0.1 * np.max(np.abs(importance_scores)),
            np.max(importance_scores) + 0.1 * np.max(np.abs(importance_scores))
        )
    
    plt.axhline(y=0., color='black', linestyle='-', linewidth=1)

    #for axis in fig.axes :
    #    axis.get_xaxis().set_visible(False)
    #    axis.get_yaxis().set_visible(False)

    plt.tight_layout()

    if save_figs :
        plt.savefig(fig_name + ".png", transparent=True, dpi=300)
        plt.savefig(fig_name + ".eps")

    plt.show()

def plot_pwm_2(pwm, figsize=(16, 2), plot_y_ticks=True, y_min=None, y_max=None, save_figs=False, fig_name="default") :
    
    fig = plt.figure(figsize=figsize)

    ax = plt.gca()
    
    height_base = 0.
    logo_height = 1.0
    
    for j in range(0, pwm.shape[0]) :
        sort_index = np.argsort(pwm[j, :])

        for ii in range(0, 4) :
            i = sort_index[ii]

            nt_prob = pwm[j, i]# * conservation[j]

            nt = ''
            if i == 0 :
                nt = 'A'
            elif i == 1 :
                nt = 'C'
            elif i == 2 :
                nt = 'G'
            elif i == 3 :
                nt = 'T'

            color = None
            if ii == 0 :
                dna_letter_at(nt, j + 0.5, height_base, nt_prob * logo_height, ax, color=color)
            else :
                prev_prob = np.sum(pwm[j, sort_index[:ii]]) * logo_height # * conservation[j]
                dna_letter_at(nt, j + 0.5, height_base + prev_prob, nt_prob * logo_height, ax, color=color)
    
    plt.sca(ax)
    plt.xticks([], [])
    plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
    
    plt.xlim((0, pwm.shape[0]))
    
    #plt.axis('off')
    
    if plot_y_ticks :
        plt.yticks(fontsize=12)
    else :
        plt.yticks([], [])
    
    if y_min is not None and y_max is not None :
        plt.ylim(y_min, y_max)
    elif y_min is not None :
        plt.ylim(y_min)
    else :
        plt.ylim(
            min(0., np.min(np.sum(pwm, axis=-1))) - 0.01 * np.max(np.abs(np.sum(pwm, axis=-1))),
            max(0., np.max(np.sum(pwm, axis=-1))) + 0.01 * np.max(np.abs(np.sum(pwm, axis=-1)))
        )
    
    print(np.min(np.sum(pwm, axis=-1)) - 0.1 * np.max(np.abs(np.sum(pwm, axis=-1))))
    print(np.max(np.sum(pwm, axis=-1)) + 0.1 * np.max(np.abs(np.sum(pwm, axis=-1))))
    
    plt.axhline(y=0., color='black', linestyle='-', linewidth=1)

    #for axis in fig.axes :
    #    axis.get_xaxis().set_visible(False)
    #    axis.get_yaxis().set_visible(False)

    plt.tight_layout()

    if save_figs :
        plt.savefig(fig_name + ".png", transparent=True, dpi=300)
        plt.savefig(fig_name + ".eps")

    plt.show()

#One-hot-encoder
class SequenceEncoder :
    
    def __init__(self, encoder_type_id, encode_dims) :
        self.encoder_type_id = encoder_type_id
        self.encode_dims = encode_dims
    
    def encode(self, seq) :
        raise NotImplementedError()
    
    def encode_inplace(self, seq, encoding) :
        raise NotImplementedError()
    
    def encode_inplace_sparse(self, seq, encoding_mat, row_index) :
        raise NotImplementedError()
    
    def decode(self, encoding) :
        raise NotImplementedError()
    
    def decode_sparse(self, encoding_mat, row_index) :
        raise NotImplementedError()
    
    def __call__(self, seq) :
        return self.encode(seq)
    
class OneHotEncoder(SequenceEncoder) :
    
    def __init__(self, seq_length, channel_map) :
        super(OneHotEncoder, self).__init__('onehot', (seq_length, len(channel_map)))
        
        self.seq_len = seq_length
        self.n_channels = len(channel_map)
        self.encode_map = channel_map
        self.decode_map = {
            val : key for key, val in channel_map.items()
        }
    
    def encode(self, seq) :
        encoding = np.zeros((self.seq_len, self.n_channels))
        
        for i in range(len(seq)) :
            if seq[i] in self.encode_map :
                channel_ix = self.encode_map[seq[i]]
                encoding[i, channel_ix] = 1.

        return encoding
    
    def encode_inplace(self, seq, encoding) :
        for i in range(len(seq)) :
            if seq[i] in self.encode_map :
                channel_ix = self.encode_map[seq[i]]
                encoding[i, channel_ix] = 1.
    
    def encode_inplace_sparse(self, seq, encoding_mat, row_index) :
        raise NotImplementError()
    
    def decode(self, encoding) :
        seq = ''
    
        for pos in range(0, encoding.shape[0]) :
            argmax_nt = np.argmax(encoding[pos, :])
            max_nt = np.max(encoding[pos, :])
            if max_nt == 1 :
                seq += self.decode_map[argmax_nt]
            else :
                seq += self.decode_map[self.n_channels - 1]

        return seq
    
    def decode_sparse(self, encoding_mat, row_index) :
        encoding = np.array(encoding_mat[row_index, :].todense()).reshape(-1, 4)
        return self.decode(encoding)


Using TensorFlow backend.
2023-04-25 23:32:31.811408: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.11.0


In [2]:

df = pd.read_csv('polyadb_features_pas_3_utr3_perturb.csv', sep='\t')

save_dict = np.load("polyadb_features_pas_3_utr3_perturb.npz")
x, m, l, c, y = save_dict['x'], save_dict['m'], save_dict['l'], save_dict['c'], save_dict['y']

print("x.shape = " + str(x.shape))
print("m.shape = " + str(m.shape))
print("l.shape = " + str(l.shape))
print("c.shape = " + str(c.shape))
print("y.shape = " + str(y.shape))


x.shape = (5267, 10, 205, 4)
m.shape = (5267, 10)
l.shape = (5267, 10)
c.shape = (5267, 10, 28)
y.shape = (5267, 10, 28)


In [3]:
#Cache/Load APARENT2 baseline score

#np.save("polyadb_features_pas_3_utr3_perturb_aparent2_all_scores", s)
s = np.load("polyadb_features_pas_3_utr3_perturb_aparent2_all_scores.npy")

print(s.shape)


(5267, 10)


In [4]:

dist_index = np.array([np.nonzero(m[i, :])[0][-1] for i in range(m.shape[0])])

dist_mask = np.zeros(m.shape)
for i in range(m.shape[0]) :
    dist_mask[i, dist_index[i]] = 1.

y_dist = []
for i in range(y.shape[0]) :
    y_dist.append(y[i:i+1, dist_index[i], :])

y_dist = np.concatenate(y_dist, axis=0)

print(y_dist.shape)


(5267, 28)


In [5]:
#Load tissue-specific PAS model and generate scores for select tissue types

subset_cell_types = np.array([
    'NT',
    'CPSF4',
    'CPSF6',
    'CSTF1',
    'CSTF3',
    'FIP1L1',
    'NUDT21',
    'RBBP6',
    'SRSF3',
    'SYMPK',
    'THOC5'
], dtype=np.object)

subset_cell_type_dict = {
    cell_type : cell_type_i for cell_type_i, cell_type in enumerate(subset_cell_types)
}


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ipykernel import kernelapp as app


In [6]:
#Define tissue-/cell- types

cell_types = np.array([
    'rpm',
    'NT',
    'CDC73',
    'CPSF1',
    'CPSF2',
    'CPSF3',
    'CPSF3L',
    'CPSF4',
    'CPSF6',
    'CSTF1',
    'CSTF3',
    'CTR9',
    'FIP1L1',
    'LEO1',
    'NUDT21',
    'PABPC1',
    'PABPN1',
    'PAF1',
    'PAPOLA',
    'PCF11',
    'RBBP6',
    'RPRD1A',
    'RPRD1B',
    'SCAF8',
    'SF3A1',
    'SRSF3',
    'SYMPK',
    'THOC5'
], dtype=np.object)

cell_type_dict = {
    cell_type : cell_type_i for cell_type_i, cell_type in enumerate(cell_types)
}


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [7]:
#PAS network definition

import keras
from keras.models import Sequential, Model, load_model

from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda
from keras.layers import Conv2D, LocallyConnected2D, MaxPooling2D, GlobalMaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, Conv1D, LocallyConnected1D, MaxPooling1D, LSTM, ConvLSTM2D, GRU, CuDNNLSTM, CuDNNGRU, BatchNormalization, LocallyConnected2D, Permute, TimeDistributed, Bidirectional
from keras.layers import Concatenate, Reshape, Softmax, Conv2DTranspose, Embedding, Multiply
from keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
from keras import regularizers
from keras import backend as K
from keras.utils.generic_utils import Progbar
from keras.layers.merge import _Merge
import keras.losses

def load_pas_network(n_cell_types=1, n_dil=6, n_channels=32, filter_size=3, filter_size_0=5, nonneg_up_to=0) :
    
    conv_0 = Conv2D(n_channels, kernel_size=(1, filter_size_0), kernel_constraint=keras.constraints.NonNeg() if nonneg_up_to > 0 else None, padding='same', activation='relu', name='pasnet_conv2d_0')
    
    drop_0 = Dropout(0.5, name='pasnet_drop_0')
    
    convs = [
        Conv2D(n_channels, kernel_size=(1, filter_size), kernel_constraint=keras.constraints.NonNeg() if i < nonneg_up_to else None, padding='same', activation='relu', dilation_rate=2**i, name='pasnet_conv2d_' + str(i)) for i in range(1, n_dil+1)
    ]
    
    drops = [
        Dropout(0.5, name='pasnet_drop_' + str(i)) for i in range(1, n_dil+1)
    ]
    
    adds = [
        Lambda(lambda x: x[0] + x[1], name='pasnet_add_' + str(i)) for i in range(1, n_dil+1)
    ]
    
    pool = Lambda(lambda x: K.mean(x, axis=(1, 2)))

    final_dense = Dense(n_cell_types*3, activation='linear', kernel_initializer='zeros', bias_initializer='zeros', name='pasnet_dense_2')
    final_reshape = Lambda(lambda x: K.reshape(x, (K.shape(x)[0], n_cell_types, 3)))
    
    def _net_func(sequence_input) :
        
        x = drop_0(conv_0(sequence_input))

        for i in range(1, n_dil+1):
            x = adds[i-1]([drops[i-1](convs[i-1](x)), x])

        pool_out = pool(x)

        final_dense_out = final_dense(pool_out)
        
        return final_reshape(final_dense_out)

    return _net_func

def _load_pas_model(model_name, n_cell_types=1) :
    
    seq_input = Input(shape=(1, 205, 4), name='seq_input')
    
    pas_net = load_pas_network(n_cell_types=n_cell_types)
    
    pred_output = pas_net(seq_input)
    
    pas_model = Model(seq_input, pred_output)
    pas_model.load_weights(model_name, by_name=True)
    pas_model.compile(loss='mean_squared_error', optimizer=keras.optimizers.SGD(0.1))
    
    return pas_model

def _predict_multi_pas(pas_model, x, batch_size=32) :
    
    y_preds = []
    for k in range(x.shape[1]) :
        y_preds.append(pas_model.predict(x=[x[:, k:k+1, ...]], batch_size=32)[:, None, ...])
    
    return np.concatenate(y_preds, axis=1)


In [8]:
#Model parameters

n_bootstraps = 5
n_cell_types = subset_cell_types.shape[0]


In [9]:
#Predict tissue model scores

tissue_models = [
    _load_pas_model("saved_models/perturb_resnet_utr3_covar_drop_ensemble_" + str(bootstrap_ix) + "_pas_model.h5", n_cell_types=n_cell_types) for bootstrap_ix in range(n_bootstraps)
]
'''
ts_ensemble = np.concatenate([_predict_multi_pas(tissue_models[bootstrap_ix], x, batch_size=32)[..., None] for bootstrap_ix in range(n_bootstraps)], axis=-1)

ts = np.mean(ts_ensemble, axis=-1)

print("ts.shape = " + str(ts.shape))
'''

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2023-04-25 23:32:35.035516: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2023-04-25 23:32:35.512707: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-25 23:32:35.513115: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla P4 major: 6 minor: 1 memoryClockRate(GHz): 1.1135
pciBusID: 0000:00:04.0
2023-04-25 23:32:35.513170: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.11.0
2023-04-25 23:32:35.516398: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.11
2023-04-25 23:32:35.517876: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
202

'\nts_ensemble = np.concatenate([_predict_multi_pas(tissue_models[bootstrap_ix], x, batch_size=32)[..., None] for bootstrap_ix in range(n_bootstraps)], axis=-1)\n\nts = np.mean(ts_ensemble, axis=-1)\n\nprint("ts.shape = " + str(ts.shape))\n'

In [10]:
#Cache/Load tissue scores
'''
np.save("polyadb_features_pas_3_utr3_perturb_resnet_covar_drop_ts_ensemble", ts_ensemble)
np.save("polyadb_features_pas_3_utr3_perturb_resnet_covar_drop_ts", ts)
'''
ts_ensemble = np.load("polyadb_features_pas_3_utr3_perturb_resnet_covar_drop_ts_ensemble.npy")
ts = np.load("polyadb_features_pas_3_utr3_perturb_resnet_covar_drop_ts.npy")

print(ts_ensemble.shape)
print(ts.shape)


(5267, 10, 11, 3, 5)
(5267, 10, 11, 3)


In [11]:
#Compute normalization statistics

cell_type_ixs = [cell_type_dict[ct] for ct in subset_cell_types.tolist()]

flat_x = np.reshape(x, (x.shape[0] * x.shape[1], 1, 205, 4))
flat_ts_ensemble = np.reshape(ts_ensemble, (x.shape[0] * x.shape[1], n_cell_types, n_bootstraps, 3))
flat_ts = np.reshape(ts, (x.shape[0] * x.shape[1], n_cell_types, 3))
flat_s = np.reshape(s, (x.shape[0] * x.shape[1],))
flat_y = np.reshape(y[:, :, cell_type_ixs], (x.shape[0] * x.shape[1], n_cell_types))
flat_gene_ind = np.reshape(np.tile(np.arange(x.shape[0])[:, None], (1, x.shape[1])), (x.shape[0] * x.shape[1],))
flat_pas_ind = np.reshape(np.tile(np.arange(x.shape[1])[None, :], (x.shape[0], 1)), (x.shape[0] * x.shape[1],))

flat_m = np.reshape(m, (x.shape[0] * x.shape[1],))
flat_dist_mask = np.reshape(dist_mask, (x.shape[0] * x.shape[1],))

flat_keep_index = np.nonzero(flat_m >= 1)[0]

flat_x = flat_x[flat_keep_index, ...]
flat_ts_ensemble = flat_ts_ensemble[flat_keep_index, ...]
flat_ts = flat_ts[flat_keep_index, ...]
flat_s = flat_s[flat_keep_index, ...]
flat_y = flat_y[flat_keep_index, ...]
flat_gene_ind = flat_gene_ind[flat_keep_index, ...]
flat_pas_ind = flat_pas_ind[flat_keep_index, ...]

flat_m = flat_m[flat_keep_index, ...]
flat_dist_mask = flat_dist_mask[flat_keep_index, ...]


In [12]:
#Construct masks for proximal/middle/distal sites

flat_prox_mask = np.array((flat_pas_ind == 0), dtype=np.float32)
flat_middle_mask = 1. - flat_dist_mask - flat_prox_mask

flat_masks = [
    flat_prox_mask,
    flat_middle_mask,
    flat_dist_mask
]


In [13]:
#Load processed (flattened) PAS IDs from original dataframe

flat_ids = np.load("polyadb_features_pas_3_utr3_perturb_flat_ids.npy", allow_pickle=True)

#Compile and flatten gene names
flat_gene_names = []
for _, row in df.iterrows() :
    flat_gene_names.extend([row['gene'] for k in range(m.shape[1]) if row['pas_exists_' + str(k)] == 1])

flat_gene_names = np.array(flat_gene_names, dtype=object)


In [14]:
#Re-load gated importance scores

flat_scores = np.load("polyadb_features_pas_3_utr3_perturb_resnet_covar_drop_flat_g_scores.npy")
flat_scores = np.tile(flat_scores, (1, 1, 1, 1, 1, 4)) * flat_x[None, None, ...]

print("flat_scores.shape = " + str(flat_scores.shape))

#Re-load gated importance scores (bootstrap replicates)

n_bootstraps_ism = 3

flat_scores_ensemble = []

for bootstrap_ix in range(n_bootstraps_ism) :
    flat_scores_curr = np.load("polyadb_features_pas_3_utr3_perturb_resnet_covar_drop_flat_g_scores_bootstrap_" + str(bootstrap_ix) + ".npy")
    flat_scores_curr = np.tile(flat_scores_curr, (1, 1, 1, 1, 1, 4)) * flat_x[None, None, ...]
    
    flat_scores_ensemble.append(flat_scores_curr)
    
    print("(bootstrap_ix = " + str(bootstrap_ix) + ") flat_scores.shape = " + str(flat_scores_curr.shape))


flat_scores.shape = (11, 3, 14764, 1, 205, 4)
(bootstrap_ix = 0) flat_scores.shape = (11, 3, 14764, 1, 205, 4)
(bootstrap_ix = 1) flat_scores.shape = (11, 3, 14764, 1, 205, 4)
(bootstrap_ix = 2) flat_scores.shape = (11, 3, 14764, 1, 205, 4)


In [15]:
#Merge proximal/middle/distal predictions and ISM scores

flat_ts_merged = np.zeros((flat_x.shape[0], n_cell_types))

flat_ts_merged[flat_prox_mask == 1., ...] = flat_ts[flat_prox_mask == 1., ..., 0]
flat_ts_merged[flat_middle_mask == 1., ...] = flat_ts[flat_middle_mask == 1., ..., 1]
flat_ts_merged[flat_dist_mask == 1., ...] = flat_ts[flat_dist_mask == 1., ..., 2]

flat_scores_merged = np.zeros((n_cell_types, flat_x.shape[0], 1, 205, 4))

flat_scores_merged[:, flat_prox_mask == 1., ...] = flat_scores[:, 0, flat_prox_mask == 1., ...]
flat_scores_merged[:, flat_middle_mask == 1., ...] = flat_scores[:, 1, flat_middle_mask == 1., ...]
flat_scores_merged[:, flat_dist_mask == 1., ...] = flat_scores[:, 2, flat_dist_mask == 1., ...]


In [16]:
#Calculate 'native' background nucleotide frequencies
s_qtl_lo = np.quantile(flat_s, q=0.0)
s_qtl_hi = np.quantile(flat_s, q=1.0)

filter_index = np.nonzero((flat_s >= s_qtl_lo) & (flat_s < s_qtl_hi))[0]

#Calculate filtered mononucleotide background frequencies
mononuc_native = np.sum(flat_x[filter_index, 0, 0:146, :], axis=(0, 1)) / np.sum(flat_x[filter_index, 0, 0:146, :])

print("mononuc_native = " + str(np.round(mononuc_native, 2).tolist()))

#Calculate 'neutral' background nucleotide frequencies
s_qtl_lo = np.quantile(flat_s, q=0.45)
s_qtl_hi = np.quantile(flat_s, q=0.55)

filter_index = np.nonzero((flat_s >= s_qtl_lo) & (flat_s < s_qtl_hi))[0]

#Calculate filtered mononucleotide background frequencies
mononuc_neutral = np.sum(flat_x[filter_index, 0, 0:146, :], axis=(0, 1)) / np.sum(flat_x[filter_index, 0, 0:146, :])

print("mononuc_neutral = " + str(np.round(mononuc_neutral, 2).tolist()))

#Calculate 'weak' background nucleotide frequencies
s_qtl_lo = np.quantile(flat_s, q=0.0)
s_qtl_hi = np.quantile(flat_s, q=0.1)

filter_index = np.nonzero((flat_s >= s_qtl_lo) & (flat_s < s_qtl_hi))[0]

#Calculate filtered mononucleotide background frequencies
mononuc_weak = np.sum(flat_x[filter_index, 0, 0:146, :], axis=(0, 1)) / np.sum(flat_x[filter_index, 0, 0:146, :])

print("mononuc_weak = " + str(np.round(mononuc_weak, 2).tolist()))

#Calculate 'stong' background nucleotide frequencies
s_qtl_lo = np.quantile(flat_s, q=0.9)
s_qtl_hi = np.quantile(flat_s, q=1.0)

filter_index = np.nonzero((flat_s >= s_qtl_lo) & (flat_s < s_qtl_hi))[0]

#Calculate filtered mononucleotide background frequencies
mononuc_stong = np.sum(flat_x[filter_index, 0, 0:146, :], axis=(0, 1)) / np.sum(flat_x[filter_index, 0, 0:146, :])

print("mononuc_stong = " + str(np.round(mononuc_stong, 2).tolist()))


mononuc_native = [0.3, 0.17, 0.18, 0.35]
mononuc_neutral = [0.31, 0.17, 0.17, 0.35]
mononuc_weak = [0.28, 0.2, 0.21, 0.31]
mononuc_stong = [0.3, 0.15, 0.16, 0.39]


In [17]:
#Code for sampling extreme sequence examples according to model scores

def _sample_sequences_by_score(cell_type_1_ix=0, cell_type_2_ix=1, pred_cell_type_2_ixs=None, score_ix=0, seq_start=0, seq_end=205, n_sequences=10, ts_diff_qtl=0.99, shuffle_region='up', bg_mode_pos='native', bg_mode_neg='native', bg_mode_neutral='native', n_shuffles=5, shuffle_window_size=9, save_name='default') :
    
    if pred_cell_type_2_ixs is None :
        pred_cell_type_2_ixs = [cell_type_2_ix]
    
    save_name = save_name + "_cell_type_1_ix_" + str(cell_type_1_ix) + "_cell_type_2_ix_" + str(cell_type_2_ix) + "_score_ix_" + str(score_ix) + "_n_sequences_" + str(n_sequences) + "_" + shuffle_region + "_window_size_" + str(window_size)
    
    if not os.path.exists("./samples/" + save_name.split("/")[0]) :
        os.makedirs("./samples/" + save_name.split("/")[0])
    
    #Get sequence encoder
    acgt_encoder = OneHotEncoder(205, {'A':0, 'C':1, 'G':2, 'T':3})
    
    #Construct backgrounds
    
    #Native mononucleotide background frequencies
    mononuc_dicts_native = [
        {
            'up' : np.zeros(4, dtype='float32'),
            'dn' : np.zeros(4, dtype='float32'),
        } for score_ixx in range(len(flat_masks))
    ]

    for i in range(flat_x.shape[0]) :
        for score_ixx in range(len(flat_masks)) :
            if flat_masks[score_ixx][i] == 1 :
                mononuc_dicts_native[score_ixx]['up'] += np.sum(flat_x[i, 0, 0:70, :], axis=0)
                mononuc_dicts_native[score_ixx]['dn'] += np.sum(flat_x[i, 0, 76:146, :], axis=0)

    for score_ixx in range(len(flat_masks)) :
        mononuc_dicts_native[score_ixx]['up'] /= np.sum(mononuc_dicts_native[score_ixx]['up'])
        mononuc_dicts_native[score_ixx]['dn'] /= np.sum(mononuc_dicts_native[score_ixx]['dn'])
    
    #Uniform frequencies
    mononuc_dicts_unif = [
        {
            'up' : 0.25 * np.ones(4, dtype='float32'),
            'dn' : 0.25 * np.ones(4, dtype='float32'),
        } for score_ixx in range(len(flat_masks))
    ]
    
    #GT-depleted frequencies
    mononuc_dicts_less_gt = [
        {
            'up' : np.array([0.30, 0.30, 0.25, 0.15], dtype='float32'),
            'dn' : np.array([0.30, 0.30, 0.25, 0.15], dtype='float32'),
        } for score_ixx in range(len(flat_masks))
    ]
    
    #GT-enriched frequencies
    mononuc_dicts_more_gt = [
        {
            'up' : np.array([0.20, 0.15, 0.25, 0.40], dtype='float32'),
            'dn' : np.array([0.20, 0.15, 0.25, 0.40], dtype='float32'),
        } for score_ixx in range(len(flat_masks))
    ]

    #Pos frequencies
    bg_dict_pos = None
    if bg_mode_pos == 'native' :
        bg_dict_pos = mononuc_dicts_native[score_ix]
    elif bg_mode_pos == 'unif' :
        bg_dict_pos = mononuc_dicts_unif[score_ix]
    elif bg_mode_pos == 'less_gt' :
        bg_dict_pos = mononuc_dicts_less_gt[score_ix]
    elif bg_mode_pos == 'more_gt' :
        bg_dict_pos = mononuc_dicts_more_gt[score_ix]

    #Neg frequencies
    bg_dict_neg = None
    if bg_mode_neg == 'native' :
        bg_dict_neg = mononuc_dicts_native[score_ix]
    elif bg_mode_neg == 'unif' :
        bg_dict_neg = mononuc_dicts_unif[score_ix]
    elif bg_mode_neg == 'less_gt' :
        bg_dict_neg = mononuc_dicts_less_gt[score_ix]
    elif bg_mode_neg == 'more_gt' :
        bg_dict_neg = mononuc_dicts_more_gt[score_ix]

    #Neutral frequencies
    bg_dict_neutral = None
    if bg_mode_neutral == 'native' :
        bg_dict_neutral = mononuc_dicts_native[score_ix]
    elif bg_mode_neutral == 'unif' :
        bg_dict_neutral = mononuc_dicts_unif[score_ix]
    elif bg_mode_neutral == 'less_gt' :
        bg_dict_neutral = mononuc_dicts_less_gt[score_ix]
    elif bg_mode_neutral == 'more_gt' :
        bg_dict_neutral = mononuc_dicts_more_gt[score_ix]
    
    #Get top N most extremely perturbed PASs (model scores, measurements and ISMs)
    flat_ts_diff = flat_ts[:, cell_type_2_ix, score_ix] - flat_ts[:, cell_type_1_ix, score_ix]
    flat_y_diff = flat_y[:, cell_type_2_ix] - flat_y[:, cell_type_1_ix]
    
    flat_diff_scores = np.sum((flat_scores[cell_type_1_ix, score_ix, :, 0, seq_start:seq_end, :] - flat_scores[cell_type_2_ix, score_ix, :, 0, seq_start:seq_end, :]) * flat_x[:, 0, seq_start:seq_end, :], axis=-1)
    
    #Sorted in ascending order of model scores (negative)
    min_ts_diff = np.quantile(flat_ts_diff[flat_masks[score_ix] == 1.], q=1.-ts_diff_qtl)

    outlier_index = np.nonzero((flat_ts_diff < min_ts_diff) & (flat_masks[score_ix] == 1.))[0]
    
    outlier_ts_diff = flat_ts_diff[outlier_index]
    outlier_y_diff = flat_y_diff[outlier_index]

    outlier_index = outlier_index[np.argsort(outlier_y_diff)][:n_sequences].tolist()

    #Save sequences to file
    with open("./samples/" + save_name + "_pos.txt", "w") as f :
        
        diff_score_col_str = ""
        for pred_cell_type_2_ix in pred_cell_type_2_ixs :
            for sc_ix in range(3) :
                diff_score_col_str += "\tdiff_ct_" + str(pred_cell_type_2_ix) + "_score_" + str(sc_ix)
        
        f.write("gene_id\tpas_id\texperiment\tseq\tusage_nt\tusage_perturb\tusage_diff" + diff_score_col_str + "\n")
        
        for i in outlier_index :
            gene_id = flat_gene_names[i]
            pas_id = flat_ids[i]
            experiment = 'wt'
            wt_seq = acgt_encoder.decode(flat_x[i, 0, seq_start:seq_end, :])
            
            y_curr_nt = round(flat_y[i, cell_type_1_ix], 4)
            y_curr_perturb = round(flat_y[i, cell_type_2_ix], 4)
            
            y_curr_diff = round(-flat_y_diff[i], 4)
            
            pred_curr = []
            for bootstrap_ix in range(n_bootstraps) :
                pred_curr.append(tissue_models[bootstrap_ix].predict(x=[
                    flat_x[i:i+1, ...]
                ], batch_size=1, verbose=False))
            
            pred_curr = np.mean(np.concatenate(pred_curr, axis=0), axis=0)
            
            diff_score_str = ""
            for pred_cell_type_2_ix in pred_cell_type_2_ixs :
                for sc_ix in range(3) :
                    diff_score_str += "\t" + str(round(pred_curr[cell_type_1_ix, sc_ix] - pred_curr[pred_cell_type_2_ix, sc_ix], 4))
            
            #Store wildtype sequence
            f.write(gene_id + "\t" + pas_id + "\t" + experiment + "\t" + wt_seq + "\t" + str(y_curr_nt) + "\t" + str(y_curr_perturb) + "\t" + str(y_curr_diff) + diff_score_str + "\n")
            
            diff_scores_curr = np.copy(flat_diff_scores[i, :])
            
            #Get mode position of importance in upstream region
            up_argmax_1 = np.argmax(diff_scores_curr[shuffle_window_size // 2:70 - shuffle_window_size // 2]) + shuffle_window_size // 2
            
            #Mark covered positions
            diff_scores_curr[max(up_argmax_1 - shuffle_window_size + 1, 0): min(up_argmax_1 + shuffle_window_size, diff_scores_curr.shape[0])] = -1e6
            
            #Select second mode
            up_argmax_2 = np.argmax(diff_scores_curr[shuffle_window_size // 2:70 - shuffle_window_size // 2]) + shuffle_window_size // 2
            
            #Get mode position of importance in downstream region
            dn_argmax_1 = np.argmax(diff_scores_curr[76 + shuffle_window_size // 2:146 - shuffle_window_size // 2]) + 76 + shuffle_window_size // 2
            
            #Mark covered positions
            diff_scores_curr[max(dn_argmax_1 - shuffle_window_size + 1, 0): min(dn_argmax_1 + shuffle_window_size, diff_scores_curr.shape[0])] = -1e6
            
            #Select second mode
            dn_argmax_2 = np.argmax(diff_scores_curr[76 + shuffle_window_size // 2:146 - shuffle_window_size // 2]) + 76 + shuffle_window_size // 2
            
            argmax_pos_1, argmax_pos_2 = -1, -1
            if shuffle_region == 'up' :
                argmax_pos_1, argmax_pos_2 = up_argmax_1, up_argmax_2
            elif shuffle_region == 'dn' or shuffle_region == 'cse' :
                argmax_pos_1, argmax_pos_2 = dn_argmax_1, dn_argmax_2
            
            bg_nuc = bg_dict_pos[shuffle_region] if shuffle_region in ['up', 'dn'] else None
            
            shuffled_seqs_1, shuffled_seqs_2 = None, None
            if shuffle_region in ['up', 'dn'] :
                shuffled_seqs_1 = _shuffle_seq(wt_seq, [[argmax_pos_1 - shuffle_window_size // 2, argmax_pos_1 + shuffle_window_size // 2 + 1]], bg_nuc, n_samples=n_shuffles, window_size=shuffle_window_size)
                shuffled_seqs_2 = _shuffle_seq(wt_seq, [[argmax_pos_2 - shuffle_window_size // 2, argmax_pos_2 + shuffle_window_size // 2 + 1]], bg_nuc, n_samples=n_shuffles, window_size=shuffle_window_size)
            else :
                flank_len = ((shuffle_window_size + 1) - 6) // 2
                
                shuffled_seqs_1 = _shuffle_cse(wt_seq, [[70-flank_len, 76+flank_len]], n_samples=n_shuffles)
                
                if argmax_pos_1 - shuffle_window_size // 2 <= 76+flank_len-1 :
                    argmax_pos_1 += (76+flank_len) - (argmax_pos_1 - shuffle_window_size // 2)
                
                shuffled_seqs_2 = _shuffle_seq(wt_seq, [[argmax_pos_1 - shuffle_window_size // 2, argmax_pos_1 + shuffle_window_size // 2 + 1]], bg_nuc, n_samples=n_shuffles, window_size=shuffle_window_size)
            
            #Construct sequences where both regions are shuffled simultaneously
            shuffled_seqs_1_and_2 = []
            for shuffled_seq_1, shuffled_seq_2 in zip(shuffled_seqs_1, shuffled_seqs_2) :
                if shuffle_region in ['up', 'dn'] :
                    if argmax_pos_1 < argmax_pos_2 :
                        shuffled_seq_1_and_2 = wt_seq[:argmax_pos_1 - shuffle_window_size // 2] + shuffled_seq_1[argmax_pos_1 - shuffle_window_size // 2:argmax_pos_1 + shuffle_window_size // 2 + 1] + wt_seq[argmax_pos_1 + shuffle_window_size // 2 + 1:argmax_pos_2 - shuffle_window_size // 2] + shuffled_seq_2[argmax_pos_2 - shuffle_window_size // 2:argmax_pos_2 + shuffle_window_size // 2 + 1] + wt_seq[argmax_pos_2 + shuffle_window_size // 2 + 1:]
                    else :
                        shuffled_seq_1_and_2 = wt_seq[:argmax_pos_2 - shuffle_window_size // 2] + shuffled_seq_2[argmax_pos_2 - shuffle_window_size // 2:argmax_pos_2 + shuffle_window_size // 2 + 1] + wt_seq[argmax_pos_2 + shuffle_window_size // 2 + 1:argmax_pos_1 - shuffle_window_size // 2] + shuffled_seq_1[argmax_pos_1 - shuffle_window_size // 2:argmax_pos_1 + shuffle_window_size // 2 + 1] + wt_seq[argmax_pos_1 + shuffle_window_size // 2 + 1:]
                else : #CSE
                    shuffled_seq_1_and_2 = wt_seq[:70] + shuffled_seq_1[70:76] + wt_seq[76:argmax_pos_1 - shuffle_window_size // 2] + shuffled_seq_2[argmax_pos_1 - shuffle_window_size // 2:argmax_pos_1 + shuffle_window_size // 2 + 1] + wt_seq[argmax_pos_1 + shuffle_window_size // 2 + 1:]
                
                shuffled_seqs_1_and_2.append(shuffled_seq_1_and_2)
            
            for shuffle_i in range(len(shuffled_seqs_1)) :
                
                shuffled_seq = shuffled_seqs_1[shuffle_i]
                
                flat_x_shuffled = acgt_encoder.encode(shuffled_seq)[None, None, ...]
                
                pred_curr = []
                for bootstrap_ix in range(n_bootstraps) :
                    pred_curr.append(tissue_models[bootstrap_ix].predict(x=[
                        flat_x_shuffled
                    ], batch_size=1, verbose=False))

                pred_curr = np.mean(np.concatenate(pred_curr, axis=0), axis=0)

                diff_score_str = ""
                for pred_cell_type_2_ix in pred_cell_type_2_ixs :
                    for sc_ix in range(3) :
                        diff_score_str += "\t" + str(round(pred_curr[cell_type_1_ix, sc_ix] - pred_curr[pred_cell_type_2_ix, sc_ix], 4))

                #Store shuffled sequence
                f.write(gene_id + "\t" + pas_id + "\t" + "shuffle_1_pos_" + str(argmax_pos_1) + "_repeat" + str(shuffle_i) + "\t" + shuffled_seq + "\t" + str(0.) + "\t" + str(0.) + "\t" + str(0.) + diff_score_str + "\n")
            
            for shuffle_i in range(len(shuffled_seqs_2)) :
                
                shuffled_seq = shuffled_seqs_2[shuffle_i]
                
                flat_x_shuffled = acgt_encoder.encode(shuffled_seq)[None, None, ...]
                
                pred_curr = []
                for bootstrap_ix in range(n_bootstraps) :
                    pred_curr.append(tissue_models[bootstrap_ix].predict(x=[
                        flat_x_shuffled
                    ], batch_size=1, verbose=False))

                pred_curr = np.mean(np.concatenate(pred_curr, axis=0), axis=0)

                diff_score_str = ""
                for pred_cell_type_2_ix in pred_cell_type_2_ixs :
                    for sc_ix in range(3) :
                        diff_score_str += "\t" + str(round(pred_curr[cell_type_1_ix, sc_ix] - pred_curr[pred_cell_type_2_ix, sc_ix], 4))

                #Store shuffled sequence
                f.write(gene_id + "\t" + pas_id + "\t" + "shuffle_2_pos_" + str(argmax_pos_2) + "_repeat" + str(shuffle_i) + "\t" + shuffled_seq + "\t" + str(0.) + "\t" + str(0.) + "\t" + str(0.) + diff_score_str + "\n")
            
            for shuffle_i in range(len(shuffled_seqs_1_and_2)) :
                
                shuffled_seq = shuffled_seqs_1_and_2[shuffle_i]
                
                flat_x_shuffled = acgt_encoder.encode(shuffled_seq)[None, None, ...]
                
                pred_curr = []
                for bootstrap_ix in range(n_bootstraps) :
                    pred_curr.append(tissue_models[bootstrap_ix].predict(x=[
                        flat_x_shuffled
                    ], batch_size=1, verbose=False))

                pred_curr = np.mean(np.concatenate(pred_curr, axis=0), axis=0)

                diff_score_str = ""
                for pred_cell_type_2_ix in pred_cell_type_2_ixs :
                    for sc_ix in range(3) :
                        diff_score_str += "\t" + str(round(pred_curr[cell_type_1_ix, sc_ix] - pred_curr[pred_cell_type_2_ix, sc_ix], 4))

                #Store shuffled sequence
                f.write(gene_id + "\t" + pas_id + "\t" + "shuffle_1_and_2_pos_" + str(argmax_pos_1) + "_and_" + str(argmax_pos_2) + "_repeat" + str(shuffle_i) + "\t" + shuffled_seq + "\t" + str(0.) + "\t" + str(0.) + "\t" + str(0.) + diff_score_str + "\n")

    #Sorted in descending order of model scores (positive)
    min_ts_diff = np.quantile(flat_ts_diff[flat_masks[score_ix] == 1.], q=ts_diff_qtl)

    outlier_index = np.nonzero((flat_ts_diff > min_ts_diff) & (flat_masks[score_ix] == 1.))[0]

    outlier_ts_diff = flat_ts_diff[outlier_index]
    outlier_y_diff = flat_y_diff[outlier_index]

    outlier_index = outlier_index[np.argsort(outlier_y_diff)[::-1]][:n_sequences].tolist()

    #Save sequences to file
    with open("./samples/" + save_name + "_neg.txt", "w") as f :
        
        diff_score_col_str = ""
        for pred_cell_type_2_ix in pred_cell_type_2_ixs :
            for sc_ix in range(3) :
                diff_score_col_str += "\tdiff_ct_" + str(pred_cell_type_2_ix) + "_score_" + str(sc_ix)
        
        f.write("gene_id\tpas_id\texperiment\tseq\tusage_nt\tusage_perturb\tusage_diff" + diff_score_col_str + "\n")
        
        for i in outlier_index :
            gene_id = flat_gene_names[i]
            pas_id = flat_ids[i]
            experiment = 'wt'
            wt_seq = acgt_encoder.decode(flat_x[i, 0, seq_start:seq_end, :])
            
            y_curr_nt = round(flat_y[i, cell_type_1_ix], 4)
            y_curr_perturb = round(flat_y[i, cell_type_2_ix], 4)
            
            y_curr_diff = round(-flat_y_diff[i], 4)
            
            pred_curr = []
            for bootstrap_ix in range(n_bootstraps) :
                pred_curr.append(tissue_models[bootstrap_ix].predict(x=[
                    flat_x[i:i+1, ...]
                ], batch_size=1, verbose=False))
            
            pred_curr = np.mean(np.concatenate(pred_curr, axis=0), axis=0)
            
            diff_score_str = ""
            for pred_cell_type_2_ix in pred_cell_type_2_ixs :
                for sc_ix in range(3) :
                    diff_score_str += "\t" + str(round(pred_curr[cell_type_1_ix, sc_ix] - pred_curr[pred_cell_type_2_ix, sc_ix], 4))
            
            #Store wildtype sequence
            f.write(gene_id + "\t" + pas_id + "\t" + experiment + "\t" + wt_seq + "\t" + str(y_curr_nt) + "\t" + str(y_curr_perturb) + "\t" + str(y_curr_diff) + diff_score_str + "\n")
            
            diff_scores_curr = np.copy(flat_diff_scores[i, :])
            
            #Get mode position of importance in upstream region
            up_argmax_1 = np.argmin(diff_scores_curr[shuffle_window_size // 2:70 - shuffle_window_size // 2]) + shuffle_window_size // 2
            
            #Mark covered positions
            diff_scores_curr[max(up_argmax_1 - shuffle_window_size + 1, 0): min(up_argmax_1 + shuffle_window_size, diff_scores_curr.shape[0])] = 1e6
            
            #Select second mode
            up_argmax_2 = np.argmin(diff_scores_curr[shuffle_window_size // 2:70 - shuffle_window_size // 2]) + shuffle_window_size // 2
            
            #Get mode position of importance in downstream region
            dn_argmax_1 = np.argmin(diff_scores_curr[76 + shuffle_window_size // 2:146 - shuffle_window_size // 2]) + 76 + shuffle_window_size // 2
            
            #Mark covered positions
            diff_scores_curr[max(dn_argmax_1 - shuffle_window_size + 1, 0): min(dn_argmax_1 + shuffle_window_size, diff_scores_curr.shape[0])] = 1e6
            
            #Select second mode
            dn_argmax_2 = np.argmin(diff_scores_curr[76 + shuffle_window_size // 2:146 - shuffle_window_size // 2]) + 76 + shuffle_window_size // 2
            
            argmax_pos_1, argmax_pos_2 = -1, -1
            if shuffle_region == 'up' :
                argmax_pos_1, argmax_pos_2 = up_argmax_1, up_argmax_2
            elif shuffle_region == 'dn' or shuffle_region == 'cse' :
                argmax_pos_1, argmax_pos_2 = dn_argmax_1, dn_argmax_2
            
            bg_nuc = bg_dict_neg[shuffle_region] if shuffle_region in ['up', 'dn'] else None
            
            shuffled_seqs_1, shuffled_seqs_2 = None, None
            if shuffle_region in ['up', 'dn'] :
                shuffled_seqs_1 = _shuffle_seq(wt_seq, [[argmax_pos_1 - shuffle_window_size // 2, argmax_pos_1 + shuffle_window_size // 2 + 1]], bg_nuc, n_samples=n_shuffles, window_size=shuffle_window_size)
                shuffled_seqs_2 = _shuffle_seq(wt_seq, [[argmax_pos_2 - shuffle_window_size // 2, argmax_pos_2 + shuffle_window_size // 2 + 1]], bg_nuc, n_samples=n_shuffles, window_size=shuffle_window_size)
            else :
                flank_len = ((shuffle_window_size + 1) - 6) // 2
                
                shuffled_seqs_1 = _shuffle_cse(wt_seq, [[70-flank_len, 76+flank_len]], n_samples=n_shuffles)
                
                if argmax_pos_1 - shuffle_window_size // 2 <= 76+flank_len-1 :
                    argmax_pos_1 += (76+flank_len) - (argmax_pos_1 - shuffle_window_size // 2)
                
                shuffled_seqs_2 = _shuffle_seq(wt_seq, [[argmax_pos_1 - shuffle_window_size // 2, argmax_pos_1 + shuffle_window_size // 2 + 1]], bg_nuc, n_samples=n_shuffles, window_size=shuffle_window_size)
            
            #Construct sequences where both regions are shuffled simultaneously
            shuffled_seqs_1_and_2 = []
            for shuffled_seq_1, shuffled_seq_2 in zip(shuffled_seqs_1, shuffled_seqs_2) :
                if shuffle_region in ['up', 'dn'] :
                    if argmax_pos_1 < argmax_pos_2 :
                        shuffled_seq_1_and_2 = wt_seq[:argmax_pos_1 - shuffle_window_size // 2] + shuffled_seq_1[argmax_pos_1 - shuffle_window_size // 2:argmax_pos_1 + shuffle_window_size // 2 + 1] + wt_seq[argmax_pos_1 + shuffle_window_size // 2 + 1:argmax_pos_2 - shuffle_window_size // 2] + shuffled_seq_2[argmax_pos_2 - shuffle_window_size // 2:argmax_pos_2 + shuffle_window_size // 2 + 1] + wt_seq[argmax_pos_2 + shuffle_window_size // 2 + 1:]
                    else :
                        shuffled_seq_1_and_2 = wt_seq[:argmax_pos_2 - shuffle_window_size // 2] + shuffled_seq_2[argmax_pos_2 - shuffle_window_size // 2:argmax_pos_2 + shuffle_window_size // 2 + 1] + wt_seq[argmax_pos_2 + shuffle_window_size // 2 + 1:argmax_pos_1 - shuffle_window_size // 2] + shuffled_seq_1[argmax_pos_1 - shuffle_window_size // 2:argmax_pos_1 + shuffle_window_size // 2 + 1] + wt_seq[argmax_pos_1 + shuffle_window_size // 2 + 1:]
                else : #CSE
                    shuffled_seq_1_and_2 = wt_seq[:70] + shuffled_seq_1[70:76] + wt_seq[76:argmax_pos_1 - shuffle_window_size // 2] + shuffled_seq_2[argmax_pos_1 - shuffle_window_size // 2:argmax_pos_1 + shuffle_window_size // 2 + 1] + wt_seq[argmax_pos_1 + shuffle_window_size // 2 + 1:]
                
                shuffled_seqs_1_and_2.append(shuffled_seq_1_and_2)
            
            for shuffle_i in range(len(shuffled_seqs_1)) :
                
                shuffled_seq = shuffled_seqs_1[shuffle_i]
                
                flat_x_shuffled = acgt_encoder.encode(shuffled_seq)[None, None, ...]
                
                pred_curr = []
                for bootstrap_ix in range(n_bootstraps) :
                    pred_curr.append(tissue_models[bootstrap_ix].predict(x=[
                        flat_x_shuffled
                    ], batch_size=1, verbose=False))

                pred_curr = np.mean(np.concatenate(pred_curr, axis=0), axis=0)

                diff_score_str = ""
                for pred_cell_type_2_ix in pred_cell_type_2_ixs :
                    for sc_ix in range(3) :
                        diff_score_str += "\t" + str(round(pred_curr[cell_type_1_ix, sc_ix] - pred_curr[pred_cell_type_2_ix, sc_ix], 4))

                #Store shuffled sequence
                f.write(gene_id + "\t" + pas_id + "\t" + "shuffle_1_pos_" + str(argmax_pos_1) + "_repeat" + str(shuffle_i) + "\t" + shuffled_seq + "\t" + str(0.) + "\t" + str(0.) + "\t" + str(0.) + diff_score_str + "\n")
            
            for shuffle_i in range(len(shuffled_seqs_2)) :
                
                shuffled_seq = shuffled_seqs_2[shuffle_i]
                
                flat_x_shuffled = acgt_encoder.encode(shuffled_seq)[None, None, ...]
                
                pred_curr = []
                for bootstrap_ix in range(n_bootstraps) :
                    pred_curr.append(tissue_models[bootstrap_ix].predict(x=[
                        flat_x_shuffled
                    ], batch_size=1, verbose=False))

                pred_curr = np.mean(np.concatenate(pred_curr, axis=0), axis=0)

                diff_score_str = ""
                for pred_cell_type_2_ix in pred_cell_type_2_ixs :
                    for sc_ix in range(3) :
                        diff_score_str += "\t" + str(round(pred_curr[cell_type_1_ix, sc_ix] - pred_curr[pred_cell_type_2_ix, sc_ix], 4))

                #Store shuffled sequence
                f.write(gene_id + "\t" + pas_id + "\t" + "shuffle_2_pos_" + str(argmax_pos_2) + "_repeat" + str(shuffle_i) + "\t" + shuffled_seq + "\t" + str(0.) + "\t" + str(0.) + "\t" + str(0.) + diff_score_str + "\n")
            
            for shuffle_i in range(len(shuffled_seqs_1_and_2)) :
                
                shuffled_seq = shuffled_seqs_1_and_2[shuffle_i]
                
                flat_x_shuffled = acgt_encoder.encode(shuffled_seq)[None, None, ...]
                
                pred_curr = []
                for bootstrap_ix in range(n_bootstraps) :
                    pred_curr.append(tissue_models[bootstrap_ix].predict(x=[
                        flat_x_shuffled
                    ], batch_size=1, verbose=False))

                pred_curr = np.mean(np.concatenate(pred_curr, axis=0), axis=0)

                diff_score_str = ""
                for pred_cell_type_2_ix in pred_cell_type_2_ixs :
                    for sc_ix in range(3) :
                        diff_score_str += "\t" + str(round(pred_curr[cell_type_1_ix, sc_ix] - pred_curr[pred_cell_type_2_ix, sc_ix], 4))

                #Store shuffled sequence
                f.write(gene_id + "\t" + pas_id + "\t" + "shuffle_1_and_2_pos_" + str(argmax_pos_1) + "_and_" + str(argmax_pos_2) + "_repeat" + str(shuffle_i) + "\t" + shuffled_seq + "\t" + str(0.) + "\t" + str(0.) + "\t" + str(0.) + diff_score_str + "\n")

    #Sorted in descending order of model scores (neutral)
    target_ts_diff = np.quantile(flat_ts_diff[flat_masks[score_ix] == 1.], q=0.5)

    outlier_index = np.nonzero(flat_masks[score_ix] == 1.)[0]

    outlier_ts_diff = flat_ts_diff[outlier_index]

    outlier_index = outlier_index[np.argsort((outlier_ts_diff - target_ts_diff)**2)][:n_sequences].tolist()

    #Save sequences to file
    with open("./samples/" + save_name + "_neutral.txt", "w") as f :
        
        diff_score_col_str = ""
        for pred_cell_type_2_ix in pred_cell_type_2_ixs :
            for sc_ix in range(3) :
                diff_score_col_str += "\tdiff_ct_" + str(pred_cell_type_2_ix) + "_score_" + str(sc_ix)
        
        f.write("gene_id\tpas_id\texperiment\tseq\tusage_nt\tusage_perturb\tusage_diff" + diff_score_col_str + "\n")
        
        for i in outlier_index :
            gene_id = flat_gene_names[i]
            pas_id = flat_ids[i]
            experiment = 'wt'
            wt_seq = acgt_encoder.decode(flat_x[i, 0, seq_start:seq_end, :])
            
            y_curr_nt = round(flat_y[i, cell_type_1_ix], 4)
            y_curr_perturb = round(flat_y[i, cell_type_2_ix], 4)
            
            y_curr_diff = round(-flat_y_diff[i], 4)
            
            pred_curr = []
            for bootstrap_ix in range(n_bootstraps) :
                pred_curr.append(tissue_models[bootstrap_ix].predict(x=[
                    flat_x[i:i+1, ...]
                ], batch_size=1, verbose=False))
            
            pred_curr = np.mean(np.concatenate(pred_curr, axis=0), axis=0)
            
            diff_score_str = ""
            for pred_cell_type_2_ix in pred_cell_type_2_ixs :
                for sc_ix in range(3) :
                    diff_score_str += "\t" + str(round(pred_curr[cell_type_1_ix, sc_ix] - pred_curr[pred_cell_type_2_ix, sc_ix], 4))
            
            #Store wildtype sequence
            f.write(gene_id + "\t" + pas_id + "\t" + experiment + "\t" + wt_seq + "\t" + str(y_curr_nt) + "\t" + str(y_curr_perturb) + "\t" + str(y_curr_diff) + diff_score_str + "\n")
            
            #Get mode position of importance in upstream region
            up_argmax = np.argmin(np.abs(0. - flat_diff_scores[i, shuffle_window_size // 2:70 - shuffle_window_size // 2])) + shuffle_window_size // 2
            
            #Get mode position of importance in downstream region
            dn_argmax = np.argmin(np.abs(0. - flat_diff_scores[i, 76 + shuffle_window_size // 2:146 - shuffle_window_size // 2])) + 76 + shuffle_window_size // 2
            
            argmax_pos = -1
            if shuffle_region == 'up' :
                argmax_pos = up_argmax
            elif shuffle_region == 'dn' :
                argmax_pos = dn_argmax
            
            bg_nuc = bg_dict_neutral[shuffle_region] if shuffle_region in ['up', 'dn'] else None
            
            shuffled_seqs = None
            if shuffle_region in ['up', 'dn'] :
                shuffled_seqs = _shuffle_seq(wt_seq, [[argmax_pos - shuffle_window_size // 2, argmax_pos + shuffle_window_size // 2 + 1]], bg_nuc, n_samples=n_shuffles, window_size=shuffle_window_size)
            else :
                flank_len = ((shuffle_window_size + 1) - 6) // 2
                shuffled_seqs = _shuffle_cse(wt_seq, [[70 - flank_len, 76 + flank_len]], n_samples=n_shuffles)
            
            for shuffle_i in range(len(shuffled_seqs)) :
                
                flat_x_shuffled = acgt_encoder.encode(shuffled_seqs[shuffle_i])[None, None, ...]
                
                pred_curr = []
                for bootstrap_ix in range(n_bootstraps) :
                    pred_curr.append(tissue_models[bootstrap_ix].predict(x=[
                        flat_x_shuffled
                    ], batch_size=1, verbose=False))

                pred_curr = np.mean(np.concatenate(pred_curr, axis=0), axis=0)

                diff_score_str = ""
                for pred_cell_type_2_ix in pred_cell_type_2_ixs :
                    for sc_ix in range(3) :
                        diff_score_str += "\t" + str(round(pred_curr[cell_type_1_ix, sc_ix] - pred_curr[pred_cell_type_2_ix, sc_ix], 4))

                #Store shuffled sequence
                f.write(gene_id + "\t" + pas_id + "\t" + "shuffle_1_pos_" + str(argmax_pos) + "_repeat" + str(shuffle_i) + "\t" + shuffled_seqs[shuffle_i] + "\t" + str(0.) + "\t" + str(0.) + "\t" + str(0.) + diff_score_str + "\n")

def _shuffle_seq(seq, ablate_regions, bg, n_samples=5, window_size=9) :
    
    nts = ['A', 'C', 'G', 'T']
    
    seqs_shuffled = []
    
    for [ablate_start, ablate_end] in ablate_regions :
        
        for sample_ix in range(n_samples) :
            rand_seq = "".join(np.random.choice(nts, size=(window_size,), p=bg, replace=True).tolist())
            ablated_seq = seq[:ablate_start] + rand_seq + seq[ablate_end:]

            seqs_shuffled.append(ablated_seq)

    return seqs_shuffled

def _shuffle_cse(seq, ablate_regions, n_samples=5) :
    
    nts = ['A', 'C', 'G', 'T']
    
    cses = [
        'TATAAA',
        'GATAAA',
        'CATAAA',
        'AGTAAA',
        'ACTAAA',
        'AATATA',
        'AATACA',
        'AATAGA',
        'AATAAT',
        'AATAAC',
    ]
    
    seqs_shuffled = []
    
    for [ablate_start, ablate_end] in ablate_regions :
        
        flank_len = ((ablate_end - ablate_start) - 6) // 2
        
        for sample_ix in range(n_samples) :
            
            rand_f1 = "".join(np.random.choice(nts, size=(flank_len,), p=[0.1, 0.35, 0.35, 0.2], replace=True).tolist())
            rand_f2 = "".join(np.random.choice(nts, size=(flank_len,), p=[0.1, 0.35, 0.35, 0.2], replace=True).tolist())
            
            rand_seq = rand_f1 + np.random.choice(cses) + rand_f2
            ablated_seq = seq[:ablate_start] + rand_seq + seq[ablate_end:]

            seqs_shuffled.append(ablated_seq)

    return seqs_shuffled


In [18]:
#Store extreme sequence samples for each perturbation based on the model scores (unif background)

#Parameter configuration
n_sequences = 100

experiment_prefix = 'unif_bg_'

window_sizes = [9, 11]
score_ixs = [0, 2]

pred_cell_type_2_ixs = [4, 6, 7, 10]

for window_size in window_sizes :
    print("-- window_size = " + str(window_size) + " --")
    
    for score_ix in score_ixs :
        print("-- score_ix = " + str(score_ix) + " --")
        
        save_name = experiment_prefix + 'window_size_' + str(window_size) + '_score_ix_' + str(score_ix) + '/apa_perturb_v3'

        cell_type_index_pairs = [
            [0, 4],
            [0, 6],
            [0, 6],
            [0, 7],
            [0, 10],
        ]

        shuffle_regions = [
            'dn',
            'up',
            'dn',
            'cse',
            'dn',
        ]
        
        bg_modes = [
            ['unif', 'unif', 'unif'],
            ['unif', 'unif', 'unif'],
            ['unif', 'unif', 'unif'],
            ['unif', 'unif', 'unif'],
            ['unif', 'unif', 'unif'],
        ]

        for pair_ix, [cell_type_1_ix, cell_type_2_ix] in enumerate(cell_type_index_pairs) :

            print("cell_type_1_ix = " + str(subset_cell_types[cell_type_1_ix]) + " vs. cell_type_2_ix = " + str(subset_cell_types[cell_type_2_ix]))

            _sample_sequences_by_score(
                cell_type_1_ix=cell_type_1_ix,
                cell_type_2_ix=cell_type_2_ix,
                pred_cell_type_2_ixs=pred_cell_type_2_ixs,
                score_ix=score_ix,
                n_sequences=n_sequences,
                ts_diff_qtl=0.96,
                shuffle_region=shuffle_regions[pair_ix],
                bg_mode_pos=bg_modes[pair_ix][0],
                bg_mode_neg=bg_modes[pair_ix][1],
                bg_mode_neutral=bg_modes[pair_ix][2],
                n_shuffles=5,
                shuffle_window_size=window_size,
                save_name=save_name,
            )


-- window_size = 9 --
-- score_ix = 0 --
cell_type_1_ix = NT vs. cell_type_2_ix = NUDT21



2023-04-25 23:33:25.384110: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.11
2023-04-25 23:33:25.621789: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.8


cell_type_1_ix = NT vs. cell_type_2_ix = NUDT21
-- score_ix = 2 --
cell_type_1_ix = NT vs. cell_type_2_ix = NUDT21
cell_type_1_ix = NT vs. cell_type_2_ix = NUDT21
-- window_size = 11 --
-- score_ix = 0 --
cell_type_1_ix = NT vs. cell_type_2_ix = NUDT21
cell_type_1_ix = NT vs. cell_type_2_ix = NUDT21
-- score_ix = 2 --
cell_type_1_ix = NT vs. cell_type_2_ix = NUDT21
cell_type_1_ix = NT vs. cell_type_2_ix = NUDT21
